<a href="https://colab.research.google.com/github/UmbrellaMalware/PythonAnalysisCourse/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Визуализационный анализ данных

Подберите на [kaggle](https://www.kaggle.com/datasets/) или в другом источнике датасет, удовлетворяющий следующим требованиям:
- количество строк - не менее 10 тысяч
- количество столбцов - не менее 10, из которых не менее 30% - числовые

Произведите предобработку данных: обработайте пустые значения, оцените выбросы.
Сформулируйте и докажите (или опровергните) 20 гипотез при помощи визуализаций имеющихся данных. Часть гипотез (но не более 25%) может касаться предобработки данных.

Постарайтесь использовать разные виды диаграмм для визуализации. Обращайте внимание на стиль и грамотность оформления. Все визуализации должны быть выполнены на Python.

Работу рекомендуется выполнять самостоятельно или в группах по 2 человека.